In [1]:
import pypsa, numpy as np
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt

Several Eastern Caribbean countries with no storage and no links, three months of hourly periods, with varying loads and generation from VRE 


In [2]:
csv_folder_name =  "Eastern_CaribbeanExtendable_rjb-12mo_v2/"
network = pypsa.Network(csv_folder_name)
scenario = "Extendable_12mo-highfuelcost/"

INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`
INFO:pypsa.io:Imported network Eastern_CaribbeanExtendable_rjb-12mo_v2 has buses, generators, loads


In [ ]:
#specify that we consider all snapshots
network.lopf(network.snapshots)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 29.38s
INFO:pypsa.linopf:Solve linear problem using Glpk solver


In [ ]:
pypsa.Network.export_to_csv_folder(network,scenario)

In [ ]:
#print the load active power (P) consumption
network.loads_t.p.plot(figsize = (15,7))

In [ ]:
#print the generator active power (P) dispatch
network.generators_t.p.plot(figsize = (15,7))
network.generators.p_nom_opt

In [ ]:
df_cap=pd.DataFrame(network.generators.p_nom_opt)
df_cap = df_cap.reset_index()
df_cap

In [ ]:
df_cap.columns = ['name','Capacity']
new = df_cap["name"].str.split(" ", n = 1, expand = True)
df_cap["Technology"]= new[1]
df_cap['Country']= new[0]
df_cap.drop(columns =["name"], inplace = True)
df_cap= df_cap.pivot(index = 'Country',columns = 'Technology', values = 'Capacity')
df_cap

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_cap.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Capacity by Technology and Country",fontsize=24)
axs.set_ylabel("Capacity [MW]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel(fontsize=20)
fig.savefig(scenario+"capacity.png", bbox_inches='tight')  

In [ ]:
df_cap_scaled = df_cap.div(df_cap.sum(axis=1), axis=0)*100
df_cap_scaled

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_cap_scaled.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Scaled Capacity by Technology and Country",fontsize=24)
axs.set_ylabel("Scaled Capacity [%]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel(fontsize=20)
axs.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.savefig(scenario+"scaled_capacity.png", bbox_inches='tight') 

In [ ]:
(network.generators.p_nom_opt*network.generators.capital_cost)

In [ ]:
df_cost=pd.DataFrame((network.generators.p_nom_opt*network.generators.capital_cost)+(network.generators_t.p.sum()* network.generators.marginal_cost))
df_cost.columns = ['cost']
df_cost = df_cost.reset_index()
df_cost

In [ ]:
new1 = df_cost["name"].str.split(" ", n = 1, expand = True)
df_cost["Technology"]= new[1]
df_cost['Country']= new[0]
df_cost.drop(columns =["name"], inplace = True)
df_cost= df_cost.pivot(index = 'Country',columns = 'Technology', values = 'cost')
df_cost

In [ ]:
df_cost.sum(axis=1)

In [ ]:
network.generators_t.p.sum()

In [ ]:
network.generators_t.p.sum().sum()

In [ ]:
df_gen=pd.DataFrame(network.generators_t.p.sum()/1000)
df_gen = df_gen.reset_index()
df_gen.columns = ['name','Electricity']
new = df_gen["name"].str.split(" ", n = 1, expand = True)
df_gen["Technology"]= new[1]
df_gen['Country']= new[0]
df_gen.drop(columns =["name"], inplace = True)
df_gen= df_gen.pivot(index = 'Country',columns = 'Technology', values = 'Electricity')

In [ ]:
df_gen

In [ ]:
df_cost.sum(axis=1)/df_gen.sum(axis=1)/1000

In [ ]:
#ax3 = df_gen.plot.bar(rot=0,stacked = True)
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_gen.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Generation by Technology and Country",fontsize=24)
axs.set_ylabel("Generation [GWh]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel(fontsize=20) 
fig.savefig(scenario+"generation.png", bbox_inches='tight') 

In [ ]:
df_gen_scaled = df_gen.div(df_gen.sum(axis=1), axis=0)*100

In [ ]:
#ax4 = df_gen_scaled.plot.bar(rot=0,stacked = True)
#ax4.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig, axs = plt.subplots(figsize=(10, 10))        # Create an empty Matplotlib Figure and Axes
df_gen_scaled.plot(ax = axs,kind = 'bar', color=['green','black', 'brown', 'red', 'orange','yellow', 'blue'], rot=0,stacked = True)                  # Use pandas to put the area plot on the prepared Figure/Axes
axs.set_title("Scaled Generation by Technology and Country",fontsize=24)
axs.set_ylabel("Scaled Generation [%]",fontsize=20)          # Do any Matplotlib customization you like
axs.set_xlabel(fontsize=20)  
axs.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
fig.savefig(scenario+"scaled_generation.png", bbox_inches='tight') 

In [ ]:
diesel_total = df_gen.diesel.sum()
diesel_total

In [ ]:
fuel_oil_total = df_gen[df_gen.index.str.contains("fuel")]

In [ ]:
Total_fossil_fuel_generation =diesel_total.sum()+fuel_oil_total.sum()
#Total_fossil_fuel_generation/1000

In [ ]:
carbon_price = 50 #USD/tonneCO2
carbon_intensity = 0.63 #tonneCO2/MWh
carbon_price*carbon_intensity #USD/MWh

In [ ]:
carbon_cost = Total_fossil_fuel_generation*carbon_intensity*carbon_price

In [ ]:
((network.generators.p_nom_opt*network.generators.capital_cost).sum()+(network.generators_t.p.sum()*network.generators.marginal_cost).sum()).sum()/(network.loads_t.p_set.sum()).sum()

In [ ]:
((network.generators.p_nom_opt*network.generators.capital_cost).sum()+(network.generators_t.p.sum()*network.generators.marginal_cost).sum()).sum()/(network.loads_t.p_set.sum().sum())

In [ ]:
#print the clearing price 
network.buses_t.marginal_price.plot(figsize = (15,7))